In [1]:
import openai, os
openai.api_key = ''
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff
import pandas as pd
from joblib import Parallel, delayed
from tqdm import tqdm

In [30]:
df_1k = pd.read_csv("/home/selvansr/capstone/wikipedia/outputs/tail2.0_withsubgraph1k_withgpt.csv")
df_1k.head()

,question,trim_subgraph,answers,api_response,gpt-3
0,What is the main subject of Self-Publishing Re...,['Self-Publishing Review instance of magazine'...,['book review'],"{\n ""choices"": [\n {\n ""finish_reason...","""Publishing and literary topics"""
1,Which conflict was 115th Infantry Division inv...,"[""division topic's main category Category:Divi...",['World War I'],"{\n ""choices"": [\n {\n ""finish_reason...","""The 115th Infantry Division was involved in ..."
2,Who developed CDC display code?,"[""character encoding topic's main category Cat...",['Control Data Corporation'],"{\n ""choices"": [\n {\n ""finish_reason...",WikiProject Computer science
3,Which military branch is 46th (South Devonshir...,"[""military unit topic's main category Category...",['British Army'],"{\n ""choices"": [\n {\n ""finish_reason...","""The 46th (South Devonshire) Regiment of Foot..."
4,What is Mashudu Tshifularo's gender?,"['human has quality self-awareness', 'human ha...",['male'],"{\n ""choices"": [\n {\n ""finish_reason...","""Male"""


In [34]:
alias = pd.read_csv("/home/selvansr/capstone/final_data/tail_1k.csv")
alias.head()

,s1,s1_ID,s2,s2_ID,s2_alias,property,property_ID,GPT-3 question,manual_question,GPT-3 answer,dpr
0,Self-Publishing Review,Q22908621,['book review'],['Q637866'],"['review', 'book review']",main subject,P921,what is the main subject of Self-Publishing Re...,What is the main subject of Self-Publishing Re...,book publishing,"[{'id': 'wiki:13657927', 'title': 'Self-publis..."
1,115th Infantry Division,Q170609,['world war i'],['Q361'],"['world war one', 'ww1', 'world war 1', 'world...",conflict,P607,what conflict did the 115th Infantry Division ...,Which conflict was 115th Infantry Division inv...,World War II,"[{'id': 'wiki:3640772', 'title': '45th Infantr..."
2,CDC display code,Q5009913,['control data corporation'],['Q1129191'],"['cdc', 'control data corporation']",developer,P178,who developed the CDC display code?,Who developed CDC display code?,dennis mccarty,"[{'id': 'wiki:12809904', 'title': 'CDC?', 'tex..."
3,46th (South Devonshire) Regiment of Foot,Q4638344,['british army'],['Q222595'],"['army of the united kingdom', 'british army']",military branch,P241,which military branch was the 46th (South Devo...,Which military branch is 46th (South Devonshir...,The British Army,"[{'id': 'wiki:9683036', 'title': '46th (South ..."
4,Mashudu Tshifularo,Q64746201,['male'],['Q6581097'],"['man', 'male person', 'male human', 'male gen...",sex or gender,P21,what is Mashudu Tshifularo's sex or gender?,What is Mashudu Tshifularo's gender?,male,"[{'id': 'wiki:18165472', 'title': 'The Adventu..."


In [36]:
import ast 
def get_document(ctx):
    return ctx[0]["text"]

alias["dpr"] = alias["dpr"].apply(ast.literal_eval)
alias["document"] = alias["dpr"].apply(get_document)
alias.head()

  3%|▎         | 29/999 [18:46<10:27:53, 38.84s/it]


,s1,s1_ID,s2,s2_ID,s2_alias,property,property_ID,GPT-3 question,manual_question,GPT-3 answer,dpr,document
0,Self-Publishing Review,Q22908621,['book review'],['Q637866'],"['review', 'book review']",main subject,P921,what is the main subject of Self-Publishing Re...,What is the main subject of Self-Publishing Re...,book publishing,"[{'id': 'wiki:13657927', 'title': 'Self-publis...",with readers. Libraries have also become invol...
1,115th Infantry Division,Q170609,['world war i'],['Q361'],"['world war one', 'ww1', 'world war 1', 'world...",conflict,P607,what conflict did the 115th Infantry Division ...,Which conflict was 115th Infantry Division inv...,World War II,"[{'id': 'wiki:3640772', 'title': '45th Infantr...","almost two years of fighting totalled 124,840 ..."
2,CDC display code,Q5009913,['control data corporation'],['Q1129191'],"['cdc', 'control data corporation']",developer,P178,who developed the CDC display code?,Who developed CDC display code?,dennis mccarty,"[{'id': 'wiki:12809904', 'title': 'CDC?', 'tex...",and an answer-key at the end. To figure out th...
3,46th (South Devonshire) Regiment of Foot,Q4638344,['british army'],['Q222595'],"['army of the united kingdom', 'british army']",military branch,P241,which military branch was the 46th (South Devo...,Which military branch is 46th (South Devonshir...,The British Army,"[{'id': 'wiki:9683036', 'title': '46th (South ...",1854 and saw action at the Battle of Alma in S...
4,Mashudu Tshifularo,Q64746201,['male'],['Q6581097'],"['man', 'male person', 'male human', 'male gen...",sex or gender,P21,what is Mashudu Tshifularo's sex or gender?,What is Mashudu Tshifularo's gender?,male,"[{'id': 'wiki:18165472', 'title': 'The Adventu...","Lastly, Mahlodi is Mashaba’s high school sweet..."


In [39]:
alias["trim_subgraph"] = df_1k["trim_subgraph"]
alias.head()

,s1,s1_ID,s2,s2_ID,s2_alias,property,property_ID,GPT-3 question,manual_question,GPT-3 answer,dpr,document,trim_subgraph
0,Self-Publishing Review,Q22908621,['book review'],['Q637866'],"['review', 'book review']",main subject,P921,what is the main subject of Self-Publishing Re...,What is the main subject of Self-Publishing Re...,book publishing,"[{'id': 'wiki:13657927', 'title': 'Self-publis...",with readers. Libraries have also become invol...,['Self-Publishing Review instance of magazine'...
1,115th Infantry Division,Q170609,['world war i'],['Q361'],"['world war one', 'ww1', 'world war 1', 'world...",conflict,P607,what conflict did the 115th Infantry Division ...,Which conflict was 115th Infantry Division inv...,World War II,"[{'id': 'wiki:3640772', 'title': '45th Infantr...","almost two years of fighting totalled 124,840 ...","[""division topic's main category Category:Divi..."
2,CDC display code,Q5009913,['control data corporation'],['Q1129191'],"['cdc', 'control data corporation']",developer,P178,who developed the CDC display code?,Who developed CDC display code?,dennis mccarty,"[{'id': 'wiki:12809904', 'title': 'CDC?', 'tex...",and an answer-key at the end. To figure out th...,"[""character encoding topic's main category Cat..."
3,46th (South Devonshire) Regiment of Foot,Q4638344,['british army'],['Q222595'],"['army of the united kingdom', 'british army']",military branch,P241,which military branch was the 46th (South Devo...,Which military branch is 46th (South Devonshir...,The British Army,"[{'id': 'wiki:9683036', 'title': '46th (South ...",1854 and saw action at the Battle of Alma in S...,"[""military unit topic's main category Category..."
4,Mashudu Tshifularo,Q64746201,['male'],['Q6581097'],"['man', 'male person', 'male human', 'male gen...",sex or gender,P21,what is Mashudu Tshifularo's sex or gender?,What is Mashudu Tshifularo's gender?,male,"[{'id': 'wiki:18165472', 'title': 'The Adventu...","Lastly, Mahlodi is Mashaba’s high school sweet...","['human has quality self-awareness', 'human ha..."


In [40]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return openai.Completion.create(**kwargs)

In [41]:
prompt = '''answer question using the document if it is relevant. Use the additional triples for more context:

Question: Where is Nelson's Pillar located?
Document: "Nelson's Pillar (also known as the Nelson Pillar or simply the Pillar) was a large granite column capped by a statue of Horatio Nelson, built in the centre of what was then Sackville Street (later renamed O'Connell Street) in Dublin, Ireland"
Additional triples: "[Nelson's Pillar instance of victory column, Nelson's Pillar country Republic of Ireland]"
Factual Answer: ["Ireland"]

Question: {}
Document: {}
Additional triples: {}
Factual Answer: '''

In [42]:
# df_new = df_new[["question", "document", "answers"]]
df_new = alias[["GPT-3 question", "document", "trim_subgraph", "s2"]]
df_new

,GPT-3 question,document,trim_subgraph,s2
0,what is the main subject of Self-Publishing Re...,with readers. Libraries have also become invol...,['Self-Publishing Review instance of magazine'...,['book review']
1,what conflict did the 115th Infantry Division ...,"almost two years of fighting totalled 124,840 ...","[""division topic's main category Category:Divi...",['world war i']
2,who developed the CDC display code?,and an answer-key at the end. To figure out th...,"[""character encoding topic's main category Cat...",['control data corporation']
3,which military branch was the 46th (South Devo...,1854 and saw action at the Battle of Alma in S...,"[""military unit topic's main category Category...",['british army']
4,what is Mashudu Tshifularo's sex or gender?,"Lastly, Mahlodi is Mashaba’s high school sweet...","['human has quality self-awareness', 'human ha...",['male']
...,...,...,...,...
994,What is the health specialty of Rachischisis?,Bariatrics Bariatrics is the branch of medicin...,['neural tube defect on focus list of Wikimedi...,['medical genetics']
995,what record label released Freddy's Favorites:...,"'youth power' film."" The film was first introd...",['album properties for this type place of publ...,['varèse sarabande']
996,What industry does Cine Q work in?,operations. QVC/CNR broadcasts live programmin...,"[""company topic's main category Category:Compa...",['entertainment']
997,what field of work does the Association for So...,Association for Software Testing The Associati...,[],['computing']


In [43]:
api_response = Parallel(n_jobs=15, backend='multiprocessing')(delayed(completion_with_backoff)(model="text-davinci-003", prompt=prompt.format(q, d, g, a)) \
                                                                for q,d,g, a in tqdm(df_new.values))

100%|██████████| 999/999 [01:10<00:00, 14.07it/s]


In [44]:
df_new["api_response"] = api_response
df_new

/home/selvansr/miniconda3/envs/sling/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,GPT-3 question,document,trim_subgraph,s2,api_response
0,what is the main subject of Self-Publishing Re...,with readers. Libraries have also become invol...,['Self-Publishing Review instance of magazine'...,['book review'],"{'id': 'cmpl-7JXmIfqZU5mijqfWIjMaLYNX2i6Ts', '..."
1,what conflict did the 115th Infantry Division ...,"almost two years of fighting totalled 124,840 ...","[""division topic's main category Category:Divi...",['world war i'],"{'id': 'cmpl-7JXmIdOsNPvPzhVUgI9mDkbHHnrTo', '..."
2,who developed the CDC display code?,and an answer-key at the end. To figure out th...,"[""character encoding topic's main category Cat...",['control data corporation'],"{'id': 'cmpl-7JXmIYAk6hdz2syyTYo5BzWBT3zn5', '..."
3,which military branch was the 46th (South Devo...,1854 and saw action at the Battle of Alma in S...,"[""military unit topic's main category Category...",['british army'],"{'id': 'cmpl-7JXmImiphvIaYhsgJqA0Rl7XUWvi0', '..."
4,what is Mashudu Tshifularo's sex or gender?,"Lastly, Mahlodi is Mashaba’s high school sweet...","['human has quality self-awareness', 'human ha...",['male'],"{'id': 'cmpl-7JXmIEH2GgH8oRNGve44axgxvzqlZ', '..."
...,...,...,...,...,...
994,What is the health specialty of Rachischisis?,Bariatrics Bariatrics is the branch of medicin...,['neural tube defect on focus list of Wikimedi...,['medical genetics'],"{'id': 'cmpl-7JXnRx3ZFoKmiTrZBSI0wi3ylUHOM', '..."
995,what record label released Freddy's Favorites:...,"'youth power' film."" The film was first introd...",['album properties for this type place of publ...,['varèse sarabande'],"{'id': 'cmpl-7JXnRa1xdHonCBikvQuVo3kgREhrs', '..."
996,What industry does Cine Q work in?,operations. QVC/CNR broadcasts live programmin...,"[""company topic's main category Category:Compa...",['entertainment'],"{'id': 'cmpl-7JXnRJE6KBro8ZvdCBVcxOTFZWZ0g', '..."
997,what field of work does the Association for So...,Association for Software Testing The Associati...,[],['computing'],"{'id': 'cmpl-7JXnRg1cPyZKJJ7fdjLqqnoG38ss5', '..."


In [48]:
def get_text(api):
    return(api["choices"][0]["text"])
    
df_new["gpt-3"] = df_new["api_response"].apply(get_text)

/home/selvansr/miniconda3/envs/sling/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [49]:
df_new.head()

,GPT-3 question,document,trim_subgraph,s2,api_response,gpt-3
0,what is the main subject of Self-Publishing Re...,with readers. Libraries have also become invol...,['Self-Publishing Review instance of magazine'...,['book review'],"{'id': 'cmpl-7JXmIfqZU5mijqfWIjMaLYNX2i6Ts', '...","[""Self-publishing""]"
1,what conflict did the 115th Infantry Division ...,"almost two years of fighting totalled 124,840 ...","[""division topic's main category Category:Divi...",['world war i'],"{'id': 'cmpl-7JXmIdOsNPvPzhVUgI9mDkbHHnrTo', '...","[""Second World War""]"
2,who developed the CDC display code?,and an answer-key at the end. To figure out th...,"[""character encoding topic's main category Cat...",['control data corporation'],"{'id': 'cmpl-7JXmIYAk6hdz2syyTYo5BzWBT3zn5', '...",William Steig
3,which military branch was the 46th (South Devo...,1854 and saw action at the Battle of Alma in S...,"[""military unit topic's main category Category...",['british army'],"{'id': 'cmpl-7JXmImiphvIaYhsgJqA0Rl7XUWvi0', '...","[""Infantry""]"
4,what is Mashudu Tshifularo's sex or gender?,"Lastly, Mahlodi is Mashaba’s high school sweet...","['human has quality self-awareness', 'human ha...",['male'],"{'id': 'cmpl-7JXmIEH2GgH8oRNGve44axgxvzqlZ', '...","\nUnfortunately, there is not enough informati..."


In [47]:
df_new.to_csv("/home/selvansr/capstone/wikipedia/outputs/tail2.0_1k_graph_dpr_gpt.csv")